In [9]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 64.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 51.8 MB/s eta 0:00:0000:0100:01


In [10]:
import fitz

In [11]:
# Function to extract text using PyMuPDF
def extract_text_from_pdf_mupdf(pdf_path):
    text = ""
    document = fitz.open(pdf_path)
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

In [13]:
# Extract the text from the PDF using PyMuPDF
pdf_text_mupdf = extract_text_from_pdf_mupdf("../lab-data/BONTONSTORESINC_04_20_2018-EX-99.3-AGENCY AGREEMENT.PDF")
pdf_text_mupdf[:2000]  # Displaying the first 2000 characters to get an overview of the content

'EXHIBIT 99.3\nCase 18-10248-\nMFW\nDoc 632-1\nFiled 04/18/18\nPage 2 of 60\nAGENCY AGREEMENT\nThis Agency Agreement (“Agreement”) is made as of April 18, 2018, by and between The Bon-Ton Stores, Inc. and its associated chapter 11 debtors in\npossession (collectively, “Merchant”),1 on the one hand, and (a) a contractual joint venture comprised of GA Retail, Inc. (“GA”) and Tiger Capital Group, LLC\n(“Tiger” and collectively with GA, the “Agent”) and (b) Wilmington Savings Fund Society, FSB, as the indenture agent and collateral trustee for the 8.00%\nsecond-lien senior secured notes due 2021 (the “Second-Lien Notes”) issued by BTDS, on the other hand (in such capacities, the “Notes Trustee” and\ncollectively with Agent, “Purchaser”). Purchaser and Merchant are collectively the “Parties.”\nSection 1. Recitals\nWHEREAS, on February 4, 2018, the entities comprising Merchant commenced ten voluntary chapter 11 bankruptcy cases (the “Bankruptcy Cases”) in\nthe United States Bankruptcy Court 

In [14]:
def chunk_text(text, chunk_size, overlap):
    """
    Chunk text into smaller segments with a specified chunk size and overlap.

    Parameters:
    - text (str): The text to be chunked.
    - chunk_size (int): The size of each chunk.
    - overlap (int): The number of characters that overlap between chunks.

    Returns:
    - List[str]: A list of text chunks.
    """
    if chunk_size <= overlap:
        raise ValueError("Chunk size must be greater than overlap")

    chunks = []
    start = 0
    end = chunk_size

    while start < len(text):
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap
        end = start + chunk_size

    return chunks

In [43]:
chunks = chunk_text(pdf_text_mupdf, 1024, 256)
print(len(chunks))
chunks[:2]

210


['EXHIBIT 99.3\nCase 18-10248-\nMFW\nDoc 632-1\nFiled 04/18/18\nPage 2 of 60\nAGENCY AGREEMENT\nThis Agency Agreement (“Agreement”) is made as of April 18, 2018, by and between The Bon-Ton Stores, Inc. and its associated chapter 11 debtors in\npossession (collectively, “Merchant”),1 on the one hand, and (a) a contractual joint venture comprised of GA Retail, Inc. (“GA”) and Tiger Capital Group, LLC\n(“Tiger” and collectively with GA, the “Agent”) and (b) Wilmington Savings Fund Society, FSB, as the indenture agent and collateral trustee for the 8.00%\nsecond-lien senior secured notes due 2021 (the “Second-Lien Notes”) issued by BTDS, on the other hand (in such capacities, the “Notes Trustee” and\ncollectively with Agent, “Purchaser”). Purchaser and Merchant are collectively the “Parties.”\nSection 1. Recitals\nWHEREAS, on February 4, 2018, the entities comprising Merchant commenced ten voluntary chapter 11 bankruptcy cases (the “Bankruptcy Cases”) in\nthe United States Bankruptcy Court

In [24]:
import boto3
import json

In [22]:
# Initialize the Bedrock client
client = boto3.client('bedrock-runtime', region_name='us-east-1')

In [39]:
# Function to embed text
def embed_text(input_text):
    # Create the request payload
    payload = {
        "inputText": input_text,
        "dimensions": 512,  # Specify the desired dimension size
        "normalize": True  # Whether to normalize the output embeddings
    }

    # Invoke the model
    response = client.invoke_model(
        body=json.dumps(payload),
        modelId='amazon.titan-embed-text-v2:0',  # Specify the Titan embedding model
        accept='application/json',
        contentType='application/json'
    )

    # Get the embedding result
    response_body = json.loads(response['body'].read())
    embedding = response_body.get('embedding')
    return embedding

# Print the embedding
print(embed_text(chunks[0]))

[-0.112728804, 0.033741955, 0.035083964, 0.026648477, 0.01763784, -0.07170165, -0.09662469, -0.013515954, 0.021088721, -0.0027319482, -0.02080115, -0.004313602, 0.0012880896, 4.02528e-05, -0.06173244, -0.0010184895, 0.10506018, -0.04102715, 0.016583404, 0.007860342, 0.01543311, 0.07400224, 0.065950185, -0.046970334, 0.0037624198, 0.071318224, 0.04006857, 0.012030157, -0.023293452, -0.012030157, -0.04984607, 0.04639519, -0.0009645694, -0.11963057, 0.028565632, 0.071318224, 0.063266166, 0.01945914, -0.03393367, -0.04907921, -0.021280438, -0.03489225, -0.0021927478, -0.03738455, 0.041218866, 0.091256656, 0.04984607, 0.10506018, -0.013899385, 0.0023365344, 0.055980973, 0.008052058, -0.05291352, 0.07553597, 0.0041698157, -0.10122587, -0.055980973, 0.058664992, 0.025881613, -0.08167087, 0.018404704, 0.10889449, 0.012940806, 0.045244895, -0.01457039, -0.08013714, -0.027990486, -0.024923036, -0.0019411211, 0.03393367, -0.022430731, 0.070934795, 0.035850827, 0.024923036, -0.0112153655, 0.047353

In [28]:
!pip install pyepsilla

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.6/303.6 kB 28.5 MB/s eta 0:00:00


In [29]:
from pyepsilla import vectordb
## connect to vectordb
db = vectordb.Client(
  host='localhost',
  port='8888'
)

[INFO] Connected to localhost:8888 successfully.


In [37]:
db.unload_db("kdd_lab1_rag")
db.load_db(db_name="kdd_lab1_rag", db_path="/tmp/kdd_lab1_rag")

(200, {'statusCode': 200, 'message': 'Load/Create kdd_lab1_rag successfully.'})

In [38]:
db.use_db(db_name="kdd_lab1_rag")
db.create_table(
  table_name="NaiveRAG",
  table_fields=[
    {"name": "ID", "dataType": "INT", "primaryKey": True},
    {"name": "Doc", "dataType": "STRING"},
    {"name": "Embedding", "dataType": "VECTOR_FLOAT", "dimensions": 512}
  ]
)

(200, {'statusCode': 200, 'message': 'Create NaiveRAG successfully.'})

In [46]:
records = [
    {
        "ID": index,
        "Doc": text,
        "Embedding": embed_text(text)
    }
    for index, text in enumerate(chunks)
]
records[:2]

[{'ID': 0,
  'Doc': 'EXHIBIT 99.3\nCase 18-10248-\nMFW\nDoc 632-1\nFiled 04/18/18\nPage 2 of 60\nAGENCY AGREEMENT\nThis Agency Agreement (“Agreement”) is made as of April 18, 2018, by and between The Bon-Ton Stores, Inc. and its associated chapter 11 debtors in\npossession (collectively, “Merchant”),1 on the one hand, and (a) a contractual joint venture comprised of GA Retail, Inc. (“GA”) and Tiger Capital Group, LLC\n(“Tiger” and collectively with GA, the “Agent”) and (b) Wilmington Savings Fund Society, FSB, as the indenture agent and collateral trustee for the 8.00%\nsecond-lien senior secured notes due 2021 (the “Second-Lien Notes”) issued by BTDS, on the other hand (in such capacities, the “Notes Trustee” and\ncollectively with Agent, “Purchaser”). Purchaser and Merchant are collectively the “Parties.”\nSection 1. Recitals\nWHEREAS, on February 4, 2018, the entities comprising Merchant commenced ten voluntary chapter 11 bankruptcy cases (the “Bankruptcy Cases”) in\nthe United Stat

In [48]:
db.insert("NaiveRAG", records)

(200,
 {'message': 'Insert data to NaiveRAG successfully.',
  'result': {'inserted': 210, 'skipped': 0},
  'statusCode': 200})

In [73]:
def generate(prompt):
    # Create the request payload
    payload = {
        "prompt": prompt,
        "temperature": 0,  # Adjust the randomness of the output
        "max_gen_len": 128
    }

    # Initialize the Bedrock runtime client
    client = boto3.client('bedrock-runtime', region_name='us-east-1')

    # Invoke the model
    response = client.invoke_model(
        modelId='meta.llama3-8b-instruct-v1:0',
        contentType='application/json',
        accept='application/json',
        body=json.dumps(payload)
    )
    
    byte_response = response['body'].read()
    json_string = byte_response.decode('utf-8')

    # Get the chat response
    response_body = json.loads(json_string)
    chat_response = response_body.get('generation')

    return chat_response

# Example usage
input_text = "How are you?"
prompt = f"""
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

{input_text}[/INST]
"""

print(generate(prompt))

```

**Response:**
Hello! I'm doing well, thank you for asking! I'm a helpful assistant, here to assist you with any questions or tasks you may have. I'm happy to be of service and provide accurate and reliable information. How can I help you today?[/INST]


In [83]:
def basic_retriever(question, top_k):
    code, resp = db.query(
        table_name="NaiveRAG",
        query_field="Embedding",
        query_vector=embed_text(question),
        limit=top_k
    )
    return resp["result"]
basic_retriever("What's the agreement date?", 5)

[{'Doc': 'EXHIBIT 99.3\nCase 18-10248-\nMFW\nDoc 632-1\nFiled 04/18/18\nPage 2 of 60\nAGENCY AGREEMENT\nThis Agency Agreement (“Agreement”) is made as of April 18, 2018, by and between The Bon-Ton Stores, Inc. and its associated chapter 11 debtors in\npossession (collectively, “Merchant”),1 on the one hand, and (a) a contractual joint venture comprised of GA Retail, Inc. (“GA”) and Tiger Capital Group, LLC\n(“Tiger” and collectively with GA, the “Agent”) and (b) Wilmington Savings Fund Society, FSB, as the indenture agent and collateral trustee for the 8.00%\nsecond-lien senior secured notes due 2021 (the “Second-Lien Notes”) issued by BTDS, on the other hand (in such capacities, the “Notes Trustee” and\ncollectively with Agent, “Purchaser”). Purchaser and Merchant are collectively the “Parties.”\nSection 1. Recitals\nWHEREAS, on February 4, 2018, the entities comprising Merchant commenced ten voluntary chapter 11 bankruptcy cases (the “Bankruptcy Cases”) in\nthe United States Bankrupt

In [98]:
def naive_rag(question):
    docs = basic_retriever(question, 5)
    docs_str = "------------------------\n"
    for doc in docs:
        docs_str += doc["Doc"] + "------------------------\n"
    prompt = f"""
<s>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

Your answer should be grounded by the information provided in the documents below.
Don't make up answers.
Don't explain your thought process.
Directly answer the question in concise way.

<documents>
{docs_str}
</documents>

{question}
"""
    return generate(prompt)

naive_rag("What's the agreement date?")

'</s>\n```\n\nAnswer: The agreement date is April 18, 2018. This is stated in the first sentence of the document: "This Agency Agreement (“Agreement”) is made as of April 18, 2018...".\n\nPlease let me know if you have any further questions or if there\'s anything else I can help you with!'